# Import data and packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/Colab Notebooks/helpers.py" .
!cp -av "/content/drive/MyDrive/Colab Notebooks/at_dataset" .

Mounted at /content/drive
'/content/drive/MyDrive/Colab Notebooks/at_dataset' -> './at_dataset'


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import glob
import cv2 as cv
from helpers import *
%load_ext autoreload
%autoreload 2


In [3]:
imgs_names = np.array(sorted(glob.glob('/content/at_dataset/images/*.tif')),dtype=object)
labels_names = np.array(sorted(glob.glob('/content/at_dataset/labels/*.tiff')),dtype=object)
idx = np.random.choice(len(imgs_names),5)
imgs = [cv.imread(name, cv.IMREAD_UNCHANGED)/(2**16-1) for name in imgs_names[idx]]
labels = [cv.imread(name, cv.IMREAD_UNCHANGED) for name in labels_names[idx]]
print(f'num data: {len(imgs)}, num labels: {len(labels)}')
X, y = segment_dataset(imgs, labels, augment=False)
# segment_dataset(imgs, labels, augment=True,store=True)
print(f'X.shape: {X.shape}, y.shape: {y.shape}')

num data: 5, num labels: 5
X.shape: (57, 572, 572), y.shape: (57, 388, 388)


In [4]:
# set up output style labels
Y = np.repeat(y[:, np.newaxis, :, :], 2, axis=1).astype(np.uint8)
Y[:,0,:,:] ^= True
print(f'X.shape: {X.shape}, Y.shape: {Y.shape}')

X.shape: (57, 572, 572), Y.shape: (57, 2, 388, 388)


In [5]:
#Accuracy 
def accuracy(pred, test_labels):
    '''
    pred: torch.tensor (result of U-net) of size [num_batches=1, 2, dim_image1, dim_image2]
    test_labels: torch.tensor (Real labels for the image) 
    '''
    '''
    Calculate the percentage of correct pixels labeled
    '''
    test_labels=test_labels.view(1,test_labels)
    label_pred=torch.argmax(pred,dim=1)
    acc= (torch.abs(label_pred-test_labels)).mean()
    return 1-acc 

In [6]:
# class DoubleConv(nn.Module):
#     """(convolution => [BN] => ReLU) * 2"""

#     def __init__(self, in_channels, out_channels, mid_channels=None):
#         super().__init__()
#         if not mid_channels:
#             mid_channels = out_channels
#         self.double_conv = nn.Sequential(
#             nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(mid_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x):
#         return self.double_conv(x)


# class Down(nn.Module):
#     """Downscaling with maxpool then double conv"""

#     def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.maxpool_conv = nn.Sequential(
#             nn.MaxPool2d(2),
#             DoubleConv(in_channels, out_channels)
#         )

#     def forward(self, x):
#         return self.maxpool_conv(x)


# class Up(nn.Module):
#     """Upscaling then double conv"""

#     def __init__(self, in_channels, out_channels, bilinear=True):
#         super().__init__()

#         # if bilinear, use the normal convolutions to reduce the number of channels
#         if bilinear:
#             self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
#             self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
#         else:
#             self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
#             self.conv = DoubleConv(in_channels, out_channels)


#     def forward(self, x1, x2):
#         x1 = self.up(x1)
#         # input is CHW
#         diffY = x2.size()[2] - x1.size()[2]
#         diffX = x2.size()[3] - x1.size()[3]

#         x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
#                         diffY // 2, diffY - diffY // 2])
#         # if you have padding issues, see
#         # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
#         # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
#         x = torch.cat([x2, x1], dim=1)
#         return self.conv(x)


# class OutConv(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(OutConv, self).__init__()
#         self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

#     def forward(self, x):
#         return self.conv(x)

In [7]:
# class U_net(nn.Module):
#     def __init__(self, n_channels, n_classes, bilinear=True):
#         super(U_net, self).__init__()
#         self.n_channels = n_channels
#         self.n_classes = n_classes
#         self.bilinear = bilinear

#         self.inc = DoubleConv(n_channels, 64)
#         self.down1 = Down(64, 128)
#         self.down2 = Down(128, 256)
#         self.down3 = Down(256, 512)
#         factor = 2 if bilinear else 1
#         self.down4 = Down(512, 1024 // factor)
#         self.up1 = Up(1024, 512 // factor, bilinear)
#         self.up2 = Up(512, 256 // factor, bilinear)
#         self.up3 = Up(256, 128 // factor, bilinear)
#         self.up4 = Up(128, 64, bilinear)
#         self.outc = OutConv(64, n_classes)

#     def forward(self, x):
#         x1 = self.inc(x)
#         x2 = self.down1(x1)
#         x3 = self.down2(x2)
#         x4 = self.down3(x3)
#         x5 = self.down4(x4)
#         x = self.up1(x5, x4)
#         x = self.up2(x, x3)
#         x = self.up3(x, x2)
#         x = self.up4(x, x1)
#         logits = self.outc(x)
#         return logits
class U_net(nn.Module):
    def __init__(self, n_channels=64):
        'U-net from the paper "Olaf Ronneberger, Philipp Fischer, and Thomas Brox": https://arxiv.org/abs/1505.04597 '
        super(U_net, self).__init__()
        self.conv1 = nn.Conv2d(1,n_channels,3)
        self.conv2 = nn.Conv2d(n_channels,n_channels,3)
        self.pool = nn.MaxPool2d(2,stride=2)
        self.conv3 = nn.Conv2d(n_channels,2*n_channels,3)
        self.conv4 = nn.Conv2d(2*n_channels,2*n_channels,3)
        self.conv5 = nn.Conv2d(2*n_channels,4*n_channels,3)
        self.conv6 = nn.Conv2d(4*n_channels,4*n_channels,3)
        self.conv7 = nn.Conv2d(4*n_channels,8*n_channels,3)
        self.conv8 = nn.Conv2d(8*n_channels,8*n_channels,3)
        self.conv9 = nn.Conv2d(8*n_channels,16*n_channels,3)
        self.conv10 = nn.Conv2d(16*n_channels,16*n_channels,3)
        #self.upconv1= nn.ConvTranspose2d(16*n_channels,16*n_channels,2)
        self.upconv1 = nn.Upsample(scale_factor=2)
        self.conv11 = nn.Conv2d(16*n_channels,8*n_channels,3)
        self.conv12 = nn.Conv2d(8*n_channels,8*n_channels,3)
        #self.upconv2= nn.ConvTranspose2d(8*n_channels,8*n_channels,2)
        self.upconv2 = nn.Upsample(scale_factor=2)
        self.conv13 = nn.Conv2d(8*n_channels,4*n_channels,3)
        self.conv14 = nn.Conv2d(4*n_channels,4*n_channels,3)
        #self.upconv3= nn.ConvTranspose2d(4*n_channels,4*n_channels,2)
        self.upconv3 = nn.Upsample(scale_factor=2)
        self.conv15 = nn.Conv2d(4*n_channels,2*n_channels,3)
        self.conv16 = nn.Conv2d(2*n_channels,2*n_channels,3)
        #self.upconv4= nn.ConvTranspose2d(2*n_channels,2*n_channels,2)
        self.upconv4 = nn.Upsample(scale_factor=2)
        self.conv17 = nn.Conv2d(2*n_channels,n_channels,3)
        self.conv18 = nn.Conv2d(n_channels,n_channels,3)
        self.conv1_1= nn.Conv2d(n_channels,2,1)
        self.activation1=nn.ReLU()
    
    def forward(self, x):
        out = self.pool(self.activation1(self.conv2(self.activation1(self.conv1(x)))))
        out = self.pool(self.activation1(self.conv4(self.activation1(self.conv3(out)))))
        out = self.pool(self.activation1(self.conv6(self.activation1(self.conv5(out)))))
        out = self.pool(self.activation1(self.conv8(self.activation1(self.conv7(out)))))
        out = self.activation1(self.conv10(self.activation1(self.conv9(out))))
        out = self.upconv1(out)
        out = self.activation1(self.conv12(self.activation1(self.conv11(out))))
        out = self.upconv2(out)
        out = self.activation1(self.conv14(self.activation1(self.conv13(out))))
        out = self.upconv3(out)
        out = self.activation1(self.conv16(self.activation1(self.conv15(out))))
        out = self.upconv4(out)
        out = self.activation1(self.conv18(self.activation1(self.conv17(out))))
        out = self.conv1_1(out)
        return out

In [10]:
#Training (adapting function from lab10)
def train(model, criterion, X, Y, optimizer, num_epochs, device):
  """
  @param model: torch.nn.Module
  @param criterion: torch.nn.modules.loss._Loss
  @param image_input: numpy.ndarray
  @param labeled_images: numpy.ndarray
  @param optimizer: torch.optim.Optimizer
  @param num_epochs: int
  """
  

  print("Starting training")
  #Cycle for epochs
  for epoch in range(num_epochs):
    # Train an epoch
    model.train()
    for i in range(X.shape[0]):
      tensor_X = torch.Tensor(X[i]).view(1,1,in_size,in_size)
      tensor_Y = torch.Tensor(Y)[i].view(1,2,out_size,out_size)
      tensor_X, tensor_Y = tensor_X.to(device), tensor_Y.to(device)

      # Evaluate the network (forward pass)
      prediction = model(tensor_X)
      loss = criterion(prediction,tensor_Y)
      
      # Compute the gradient
      optimizer.zero_grad()
      loss.backward()

      # Update the parameters of the model with a gradient step
      optimizer.step()

The loss function used to train the network is CrossEntropy as described in the paper. 


In [11]:
num_epochs=10
learning_rate=0.001
# batch_size = 1

n, in_size, out_size = X.shape[0], 572, 388
print(n)

if not torch.cuda.is_available():
  raise Exception("Things will go much quicker if you enable a GPU in Colab under 'Runtime / Change Runtime Type'")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# dataset = TensorDataset(torch.Tensor(X).view(n,1,1,in_size,in_size),torch.Tensor(Y).view(n,1,2,out_size,out_size)) # create your datset
# dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size) # create your dataloader

model=U_net().to(device)

optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
criterion = nn.BCEWithLogitsLoss() 

train(model, criterion, X, Y, optimizer, num_epochs, device) 

57
Starting training


In [ ]:
model.eval()